# 数据准备

In [1]:
import pandas as pd
import numpy as np
 
np.random.seed(24)
df = pd.DataFrame({'A':np.linspace(1,10,10)})
df = pd.concat([df,pd.DataFrame(np.random.randn(10,4),columns=list('BCDE'))],axis=1)
df.iloc[3,3] = np.nan
df.iloc[0,2] = np.nan
df

,A,B,C,D,E
0,1.0,1.329212,NaN,-0.316280,-0.990810
1,2.0,-1.070816,-1.438713,0.564417,0.295722
2,3.0,-1.626404,0.219565,0.678805,1.889273
3,4.0,0.961538,0.104011,NaN,0.850229
4,5.0,1.453425,1.057737,0.165562,0.515018
5,6.0,-1.336936,0.562861,1.392855,-0.063328
6,7.0,0.121668,1.207603,-0.002040,1.627796
7,8.0,0.354493,1.037528,-0.385684,0.519818
8,9.0,1.686583,-1.325963,1.428984,-2.089354
9,10.0,-0.129820,0.631523,-0.586538,0.290720


# 内置方法设置样式

## 高亮行/列 max,min,null值

In [2]:
# 高亮 行/列 最大值（axis 1:行 ，0：列）
df.style.highlight_max(axis=1)
# # 高亮 行/列 最小值
df.style.highlight_min(axis=1)
# # 高亮 行/列 空值
df.style.highlight_null(null_color='red')

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


# 自定义方法设置样式
- Style.applymap 作用于DataFrame中的全部元素。
- Style.apply 作用于DataFrame中的指定行/列元素（列axis=0,行：axis=1）

## 编写自定义函数

In [3]:
# 将负数涂成红色，将正数涂成黑色
def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color_negative_red
# 突出显示每列中的最大值的函数
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

## 应用自定义函数
- 当使用多个函数处理一个DF对象时，可以使用`链式操作`。
- df.style.applymap().apply()

In [4]:
df.style.applymap(color_negative_red).apply(highlight_max)

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


# 样式处理工具

## 切片选择列进行样式处理
- subset 参数控制作用的行列。
- subset 传入的参数类似DataFrame的切片：
- 元组(row_indexer, column_indexer)

In [24]:
df.style.apply(highlight_max, subset=['B', 'C', 'D'])

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


## 值格式设置

### 自定义值的显示样式
- 使用Styler.format控制。类似字符串输出方法。

In [23]:
df.style.format("{:.2%}")

,A,B,C,D,E
0,100.00%,132.92%,nan%,-31.63%,-99.08%
1,200.00%,-107.08%,-143.87%,56.44%,29.57%
2,300.00%,-162.64%,21.96%,67.88%,188.93%
3,400.00%,96.15%,10.40%,nan%,85.02%
4,500.00%,145.34%,105.77%,16.56%,51.50%
5,600.00%,-133.69%,56.29%,139.29%,-6.33%
6,700.00%,12.17%,120.76%,-0.20%,162.78%
7,800.00%,35.45%,103.75%,-38.57%,51.98%
8,900.00%,168.66%,-132.60%,142.90%,-208.94%
9,1000.00%,-12.98%,63.15%,-58.65%,29.07%


### 自定义指定列的值的显示样式

#### 法一：常规方法

In [24]:
df.style.format({'B': "{:0<4.0f}", 'D': '{:+.2f}'})

,A,B,C,D,E
0,1.000000,1000,nan,-0.32,-0.990810
1,2.000000,-100,-1.438713,+0.56,0.295722
2,3.000000,-200,0.219565,+0.68,1.889273
3,4.000000,1000,0.104011,+nan,0.850229
4,5.000000,1000,1.057737,+0.17,0.515018
5,6.000000,-100,0.562861,+1.39,-0.063328
6,7.000000,0000,1.207603,-0.00,1.627796
7,8.000000,0000,1.037528,-0.39,0.519818
8,9.000000,2000,-1.325963,+1.43,-2.089354
9,10.000000,-000,0.631523,-0.59,0.290720


#### 法二：lambda函数

In [26]:
df.style.format({"B": lambda x: "±{:.2f}".format(abs(x))})
#df2.style.format({'总抚养比': "{:.2f}%", '少儿抚养比': '{:.2f}%', '老年抚养比': '{:.2f}%'})

,A,B,C,D,E
0,1.000000,±1.33,nan,-0.316280,-0.990810
1,2.000000,±1.07,-1.438713,0.564417,0.295722
2,3.000000,±1.63,0.219565,0.678805,1.889273
3,4.000000,±0.96,0.104011,nan,0.850229
4,5.000000,±1.45,1.057737,0.165562,0.515018
5,6.000000,±1.34,0.562861,1.392855,-0.063328
6,7.000000,±0.12,1.207603,-0.002040,1.627796
7,8.000000,±0.35,1.037528,-0.385684,0.519818
8,9.000000,±1.69,-1.325963,1.428984,-2.089354
9,10.000000,±0.13,0.631523,-0.586538,0.290720


## 撇开值设计画布样式

In [43]:
df.style.set_properties(**{'background-color': 'white','color': 'green'})

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


## 共享样式
- 假设为DataFrame建立了一个样式，想将相同样式应用于第二个DataFrame。使用df1.style.export导出样式。

In [44]:
df2 = -df
style1 = df.style.applymap(color_negative_red)
style1

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [45]:
style2 = df2.style
style2.use(style1.export())
style2

,A,B,C,D,E
0,-1.000000,-1.329212,nan,0.316280,0.990810
1,-2.000000,1.070816,1.438713,-0.564417,-0.295722
2,-3.000000,1.626404,-0.219565,-0.678805,-1.889273
3,-4.000000,-0.961538,-0.104011,nan,-0.850229
4,-5.000000,-1.453425,-1.057737,-0.165562,-0.515018
5,-6.000000,1.336936,-0.562861,-1.392855,0.063328
6,-7.000000,-0.121668,-1.207603,0.002040,-1.627796
7,-8.000000,-0.354493,-1.037528,0.385684,-0.519818
8,-9.000000,-1.686583,1.325963,-1.428984,2.089354
9,-10.000000,0.129820,-0.631523,0.586538,-0.290720


## 设置标题

In [46]:
df.style.set_caption('Colormaps, with a caption.').background_gradient(cmap=cm)

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


## 缺失值处理

### 缺失值填充并高亮

In [47]:

(df.style
   .set_na_rep("FAIL")
   .format(None, na_rep="PASS", subset=["D"])
   .highlight_null("yellow"))

,A,B,C,D,E
0,1.000000,1.329212,FAIL,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,PASS,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


### 缺失值填充

In [27]:
df.style.format("{:.2%}", na_rep="空值")

,A,B,C,D,E
0,100.00%,132.92%,空值,-31.63%,-99.08%
1,200.00%,-107.08%,-143.87%,56.44%,29.57%
2,300.00%,-162.64%,21.96%,67.88%,188.93%
3,400.00%,96.15%,10.40%,空值,85.02%
4,500.00%,145.34%,105.77%,16.56%,51.50%
5,600.00%,-133.69%,56.29%,139.29%,-6.33%
6,700.00%,12.17%,120.76%,-0.20%,162.78%
7,800.00%,35.45%,103.75%,-38.57%,51.98%
8,900.00%,168.66%,-132.60%,142.90%,-208.94%
9,1000.00%,-12.98%,63.15%,-58.65%,29.07%


In [29]:
df.style.highlight_max().format(None, na_rep="-")

,A,B,C,D,E
0,1.000000,1.329212,-,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


## 隐藏索引/列显示

In [48]:
# 隐藏索引不显示（视图模式）
df.style.hide_index()

A,B,C,D,E
1.000000,1.329212,nan,-0.316280,-0.990810
2.000000,-1.070816,-1.438713,0.564417,0.295722
3.000000,-1.626404,0.219565,0.678805,1.889273
4.000000,0.961538,0.104011,nan,0.850229
5.000000,1.453425,1.057737,0.165562,0.515018
6.000000,-1.336936,0.562861,1.392855,-0.063328
7.000000,0.121668,1.207603,-0.002040,1.627796
8.000000,0.354493,1.037528,-0.385684,0.519818
9.000000,1.686583,-1.325963,1.428984,-2.089354
10.000000,-0.129820,0.631523,-0.586538,0.290720


In [53]:
#隐藏指定列不显示（视图模式）
x = df.style.hide_columns(['C','D'])
x

,A,B,E
0,1.000000,1.329212,-0.990810
1,2.000000,-1.070816,0.295722
2,3.000000,-1.626404,1.889273
3,4.000000,0.961538,0.850229
4,5.000000,1.453425,0.515018
5,6.000000,-1.336936,-0.063328
6,7.000000,0.121668,1.627796
7,8.000000,0.354493,0.519818
8,9.000000,1.686583,-2.089354
9,10.000000,-0.129820,0.290720


In [54]:
x.columns

Index(['A', 'B', 'C', 'D', 'E'], dtype='object')

# 设计"热力图"样式

## 指定颜色

In [25]:
import seaborn as sns
# 设置指定颜色的渐变色域
cm = sns.light_palette("red", as_cmap=True)
df.style.background_gradient(cmap=cm)

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


## 不指定颜色(默认蓝色)

In [31]:
df.loc[:4].style.background_gradient()

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018


## 限定色谱百分比范围。

In [36]:
# Compress the color range
(df.loc[:4]
     .style
     .background_gradient(cmap='viridis', low=.5, high=2)
     .highlight_null('red'))

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018


,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


# 数据中包含“条形图”

## 常规模式

In [43]:
df.style.bar(subset=['A','B'],color='#d65f5f')

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


## 控制对齐方式:
- left: 最小值从单元格的左侧开始。
- zero: 零值位于单元格的中心。
- mid: t单元格的中心在（max-min）/ 2，或者如果值全为负（正），则零对齐于单元格的右（左）。

In [46]:
df.style.bar(subset=['A','B'],align='mid',color=['#d65f5f','#5fba7d'])

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


# 其他样式

In [50]:
df.style.applymap(color_negative_red).apply(highlight_max).set_precision(2)

,A,B,C,D,E
0,1.00,1.33,nan,-0.32,-0.99
1,2.00,-1.07,-1.44,0.56,0.30
2,3.00,-1.63,0.22,0.68,1.89
3,4.00,0.96,0.10,nan,0.85
4,5.00,1.45,1.06,0.17,0.52
5,6.00,-1.34,0.56,1.39,-0.06
6,7.00,0.12,1.21,-0.00,1.63
7,8.00,0.35,1.04,-0.39,0.52
8,9.00,1.69,-1.33,1.43,-2.09
9,10.00,-0.13,0.63,-0.59,0.29


## 动态样式引用

- `slector="tr"`:表格中每行都是一个tr标签。
- `slector="tr:hover"`:鼠标划过每个tr标签时的，响应事件。

In [21]:
from IPython.display import HTML
 
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",props=[("background-color","%s"%hover_color)])
 
styles = [hover(),
          dict(selector="tr",props=[("transition","1s")]),
          dict(selector="tr:hover",props=[("cursor","pointer"),
                                         ("transform","translate(50%)")]),
          dict(selector="th",props=[("font-size","150%"),
                                     ("text-align","center")]),
          dict(selector="caption",props=[("caption-side","top")])
]
html = (df.style.set_table_styles(styles).set_caption("Hover to highlight."))
html

,A,B,C,D,E
0,1.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,nan,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720
